# Trexquant Interview Project (The Hangman Game)

* Copyright Trexquant Investment LP. All Rights Reserved. 
* Redistribution of this question without written consent from Trexquant is prohibited

## Instruction:
For this coding test, your mission is to write an algorithm that plays the game of Hangman through our API server. 

When a user plays Hangman, the server first selects a secret word at random from a list. The server then returns a row of underscores (space separated)—one for each letter in the secret word—and asks the user to guess a letter. If the user guesses a letter that is in the word, the word is redisplayed with all instances of that letter shown in the correct positions, along with any letters correctly guessed on previous turns. If the letter does not appear in the word, the user is charged with an incorrect guess. The user keeps guessing letters until either (1) the user has correctly guessed all the letters in the word
or (2) the user has made six incorrect guesses.

You are required to write a "guess" function that takes current word (with underscores) as input and returns a guess letter. You will use the API codes below to play 1,000 Hangman games. You have the opportunity to practice before you want to start recording your game results.

Your algorithm is permitted to use a training set of approximately 250,000 dictionary words. Your algorithm will be tested on an entirely disjoint set of 250,000 dictionary words. Please note that this means the words that you will ultimately be tested on do NOT appear in the dictionary that you are given. You are not permitted to use any dictionary other than the training dictionary we provided. This requirement will be strictly enforced by code review.

You are provided with a basic, working algorithm. This algorithm will match the provided masked string (e.g. a _ _ l e) to all possible words in the dictionary, tabulate the frequency of letters appearing in these possible words, and then guess the letter with the highest frequency of appearence that has not already been guessed. If there are no remaining words that match then it will default back to the character frequency distribution of the entire dictionary.

This benchmark strategy is successful approximately 18% of the time. Your task is to design an algorithm that significantly outperforms this benchmark.

import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [1]:
with open("words_250000_train.txt", "r") as f:
    word_list = [line.strip().lower() for line in f if line.strip().isalpha()]
import random

max_length = max(len(word) for word in word_list)
print(f"The maximum word length is {max_length}")

The maximum word length is 29


In [3]:
import random

def simulate_multilabel_hangman_state(word, num_revealed):
    word = word.lower()
    unique_letters = list(set(word))

    if num_revealed >= len(unique_letters):
        return None  # no letters left to guess

    revealed_letters = random.sample(unique_letters, num_revealed)
    
    pattern = ''.join(c if c in revealed_letters else '_' for c in word)
    guessed_letters = revealed_letters.copy()
    
    next_letters = list(set(c for c in word if c not in guessed_letters))
    if not next_letters:
        return None

    return {
        'word': word,
        'pattern': pattern,
        'guessed_letters': guessed_letters,
        'next_letters': next_letters  # ← multi-label
    }

def build_multilabel_dataset(word_list, max_samples_per_level=1):
    dataset = []
    for word in word_list:
        unique_letter_count = len(set(word))
        for num_revealed in range(unique_letter_count):  # from 0 up to all but one
            for _ in range(max_samples_per_level):
                sample = simulate_multilabel_hangman_state(word, num_revealed)
                if sample:
                    dataset.append(sample)
    return dataset    
dataset = build_multilabel_dataset(word_list)

# Preview a few samples
for d in dataset[:5]:
    print(d)
len(dataset)

{'word': 'aaa', 'pattern': '___', 'guessed_letters': [], 'next_letters': ['a']}
{'word': 'aaaaaa', 'pattern': '______', 'guessed_letters': [], 'next_letters': ['a']}
{'word': 'aaas', 'pattern': '____', 'guessed_letters': [], 'next_letters': ['a', 's']}
{'word': 'aaas', 'pattern': '___s', 'guessed_letters': ['s'], 'next_letters': ['a']}
{'word': 'aachen', 'pattern': '______', 'guessed_letters': [], 'next_letters': ['c', 'a', 'e', 'n', 'h']}


1681209

In [4]:
from torch.utils.data import Dataset
import torch

class HangmanDataset(Dataset):
    def __init__(self, samples, max_len=30):
        self.samples = samples
        self.max_len = max_len

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        pattern_str = sample['pattern'].replace(" ", "")
        guessed_letters = set(sample['guessed_letters'])
        next_letters = sample['next_letters']  # list of next possible letters

        # Encoding pattern: '_' → 0, 'a'-'z' → 1-26
        pattern_encoded = [
            0 if c == '_' else ord(c) - ord('a') + 1 for c in pattern_str
        ]
        if len(pattern_encoded) < self.max_len:
            pattern_encoded += [27] * (self.max_len - len(pattern_encoded))  # Padding idx = 27
        else:
            pattern_encoded = pattern_encoded[:self.max_len]

        # Encoding guessed letters
        guessed_vec = [1 if chr(ord('a') + i) in guessed_letters else 0 for i in range(26)]

        # Encoding multi-label target vector
        label_vec = [0] * 26
        for ch in next_letters:
            idx = ord(ch) - ord('a')
            if 0 <= idx < 26:
                label_vec[idx] = 1

        return (
            torch.tensor(pattern_encoded, dtype=torch.long),     # [max_len]
            torch.tensor(guessed_vec, dtype=torch.float),        # [26]
            torch.tensor(label_vec, dtype=torch.float)           # [26]
        )


In [5]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

# Split the raw dataset
train_samples, test_samples = train_test_split(dataset, test_size=0.01, random_state=42)

# Wrap into custom Dataset
train_dataset = HangmanDataset(train_samples)
test_dataset  = HangmanDataset(test_samples)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn

class MultiLabelHangmanTransformer(nn.Module):
    def __init__(self, vocab_size=28, d_model=256, max_len=30, nhead=4, num_layers=2, dropout=0.1):
        super().__init__()
        self.max_len = max_len
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=vocab_size - 1)
        self.position_embedding = nn.Embedding(max_len + 1, d_model)  # +1 for guess token
        self.token_type_embedding = nn.Embedding(2, d_model)  # 0: guess, 1: pattern

        self.guess_proj = nn.Linear(26, d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.output_layer = nn.Linear(d_model, 26)

    def forward(self, pattern_tokens, guessed_mask):
        # pattern_tokens: [B, L]
        # guessed_mask: [B, 26]

        B, L = pattern_tokens.shape
        device = pattern_tokens.device

        # Embedding lookup
        pattern_embed = self.embedding(pattern_tokens)
        pos_ids = torch.arange(1, L + 1, device=device).unsqueeze(0).expand(B, L)
        pos_embed = self.position_embedding(pos_ids)
        token_type_embed = self.token_type_embedding(torch.ones_like(pos_ids))  # pattern = 1
        pattern_embed = pattern_embed + pos_embed + token_type_embed

        # Guess token
        guess_context = self.guess_proj(guessed_mask).unsqueeze(1)  # [B, 1, D]
        guess_pos = self.position_embedding(torch.zeros(B, 1, dtype=torch.long, device=device))  # pos=0
        guess_token_type = self.token_type_embedding(torch.zeros(B, 1, dtype=torch.long, device=device))  # type=0
        guess_embed = guess_context + guess_pos + guess_token_type

        # Concatenate
        x = torch.cat([guess_embed, pattern_embed], dim=1)  # [B, L+1, D]

        # Transformer forward
        x = self.transformer(x)
        x = self.norm(x[:, 0, :])  # CLS-style: use guess token
        return self.output_layer(x)  # raw logits (use with BCEWithLogitsLoss)



In [7]:
def encode_pattern(pattern_str, max_len=30):
    # '_' => 0, 'a'-'z' => 1-26, pad => 27
    cleaned = pattern_str.replace(" ", "")
    encoded = [(0 if c == '_' else ord(c) - ord('a') + 1) for c in cleaned]
    if len(encoded) < max_len:
        encoded += [27] * (max_len - len(encoded))
    return torch.tensor(encoded[:max_len], dtype=torch.long)

def encode_guessed(guessed_set):
    return torch.tensor([1 if chr(ord('a') + i) in guessed_set else 0 for i in range(26)], dtype=torch.float)


In [8]:
class MultiLabelHangmanTransformer(nn.Module):
    def __init__(self, vocab_size=28, d_model=256, max_len=30, nhead=4, num_layers=4, dropout=0.1):
        super().__init__()
        self.max_len = max_len
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=vocab_size - 1)
        self.position_embedding = nn.Embedding(max_len + 1, d_model)  # +1 for guess token
        self.token_type_embedding = nn.Embedding(2, d_model)  # 0: guess, 1: pattern

        self.guess_proj = nn.Linear(26, d_model)

        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dropout=dropout, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.norm = nn.LayerNorm(d_model)
        self.output_layer = nn.Linear(d_model, 26)

    def forward(self, pattern_tokens, guessed_mask):
        # pattern_tokens: [B, L]
        # guessed_mask: [B, 26]

        B, L = pattern_tokens.shape
        device = pattern_tokens.device

        # Embedding lookup
        pattern_embed = self.embedding(pattern_tokens)
        pos_ids = torch.arange(1, L + 1, device=device).unsqueeze(0).expand(B, L)
        pos_embed = self.position_embedding(pos_ids)
        token_type_embed = self.token_type_embedding(torch.ones_like(pos_ids))  # pattern = 1
        pattern_embed = pattern_embed + pos_embed + token_type_embed

        # Guess token
        guess_context = self.guess_proj(guessed_mask).unsqueeze(1)  # [B, 1, D]
        guess_pos = self.position_embedding(torch.zeros(B, 1, dtype=torch.long, device=device))  # pos=0
        guess_token_type = self.token_type_embedding(torch.zeros(B, 1, dtype=torch.long, device=device))  # type=0
        guess_embed = guess_context + guess_pos + guess_token_type

        # Concatenate
        x = torch.cat([guess_embed, pattern_embed], dim=1)  # [B, L+1, D]

        # Transformer forward
        x = self.transformer(x)
        x = self.norm(x[:, 0, :])  # CLS-style: use guess token
        return self.output_layer(x)  # raw logits (use with BCEWithLogitsLoss)

In [15]:
import torch
from tqdm import tqdm

# if torch.backends.mps.is_available():
#     device = torch.device("mps")
# elif torch.cuda.is_available():
#     device = torch.device("cuda")
# else:
#     device = torch.device("cpu")

device = torch.device("cpu")
model = MultiLabelHangmanTransformer().to(device)
criterion = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [17]:
import torch.optim as optim
optimizer = optim.Adam(
    model.parameters(),
    lr=1e-4,            # typical safe starting LR
    betas=(0.9, 0.999), # default values
    weight_decay=1e-5   # small weight decay for generalization
)

In [19]:
model.train()

num_epochs = 10

for epoch in range(num_epochs):
    total_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

    for pattern_batch, guessed_batch, label_batch in pbar:
        pattern_batch = pattern_batch.to(device)
        guessed_batch = guessed_batch.to(device)
        label_batch = label_batch.to(device)

        optimizer.zero_grad()
        outputs = model(pattern_batch, guessed_batch)  # [B, 26]
        loss = criterion(outputs, label_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pbar.set_postfix(loss=loss.item())

    avg_loss = total_loss / len(train_loader)
    print(f"✅ Epoch {epoch+1} finished - Average Loss: {avg_loss:.4f}")


Epoch 1:   0%|                 | 13/26007 [00:03<1:43:22,  4.19it/s, loss=0.414]


KeyboardInterrupt: 

In [21]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score
from tqdm import tqdm
import numpy as np

def evaluate(model, test_loader, threshold=0.5, device='cpu'):
    model.eval()

    y_true = []
    y_pred = []

    with torch.no_grad():
        pbar = tqdm(test_loader, desc="Evaluating")

        for pattern_batch, guessed_batch, label_batch in pbar:
            pattern_batch = pattern_batch.to(device)
            guessed_batch = guessed_batch.to(device)
            label_batch = label_batch.to(device)

            outputs = model(pattern_batch, guessed_batch)  # [B, 26]
            probs = outputs.cpu().numpy()
            targets = label_batch.cpu().numpy()

            preds = (probs >= threshold).astype(int)

            y_true.append(targets)
            y_pred.append(preds)

    y_true = np.vstack(y_true)
    y_pred = np.vstack(y_pred)

    precision = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='macro', zero_division=0)

    print(f"\n📊 Evaluation Results")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    
evaluate(model, train_loader, threshold=0.5, device=device)

In [79]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections

try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [149]:
class HangmanAPI(object):
    def __init__(self, access_token=None, session=None, timeout=None):
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)        
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = []
        
    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com']

        data = {link: 0 for link in links}

        for link in links:

            requests.get(link)

            for i in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        link = sorted(data.items(), key=lambda x: x[1])[0][0]
        link += '/trexsim/hangman'
        return link

    def guess(self, word_pattern):
        """
        Try model's top 3 guesses. If all used, fallback to frequency-based method using current dictionary.
        """
        import torch
        import re
        import collections
    
        # --------- Step 1: Model-based Guessing ---------
        clean_pattern = word_pattern[::2]
        pattern_enc = [0 if c == '_' else ord(c) - ord('a') + 1 for c in clean_pattern]
        pattern_enc = pattern_enc[:30] + [27] * (30 - len(pattern_enc))  # pad to 30
    
        guessed_mask = [1 if chr(ord('a') + i) in self.guessed_letters else 0 for i in range(26)]
    
        pattern_tensor = torch.tensor([pattern_enc], dtype=torch.long)
        guessed_tensor = torch.tensor([guessed_mask], dtype=torch.float32)    
        model.eval()
        with torch.no_grad():
            logits = model(pattern_tensor, guessed_tensor)  # [1, 26]
            probs = torch.sigmoid(logits).squeeze(0).cpu().numpy()
    
        sorted_indices = probs.argsort()[::-1]  # descending by confidence
    
        top_model_guesses = 0
        for idx in sorted_indices:
            letter = chr(ord('a') + idx)
            if letter not in self.guessed_letters:
                if top_model_guesses < 3:
                    return letter
                else:
                    break
            top_model_guesses += 1
    
        # --------- Step 2: Frequency-based fallback ---------
        # Clean and regexify the word pattern
        clean_word = word_pattern[::2].replace("_", ".")
        len_word = len(clean_word)
    
        # Update current dictionary
        current_dictionary = self.current_dictionary
        new_dictionary = []
        for dict_word in current_dictionary:
            if len(dict_word) != len_word:
                continue
            if re.match(clean_word, dict_word):
                new_dictionary.append(dict_word)
        self.current_dictionary = new_dictionary
    
        # Count frequencies in remaining plausible words
        full_dict_string = "".join(new_dictionary)
        c = collections.Counter(full_dict_string)
        sorted_letter_count = c.most_common()
    
        for letter, _ in sorted_letter_count:
            if letter not in self.guessed_letters:
                return letter
    
        # --------- Step 3: Global frequency fallback ---------
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter
    
        # If all letters guessed
        return None


    ##########################################################
    # You'll likely not need to modify any of the code below #
    ##########################################################
    
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary
                
    def start_game(self, practice=True, verbose=True):
        # reset guessed letters to empty set and current plausible dictionary to the full dictionary
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
                         
        response = self.request("/new_game", {"practice":practice})
        if response.get('status')=="approved":
            game_id = response.get('game_id')
            word = response.get('word')
            tries_remains = response.get('tries_remains')
            if verbose:
                print("Successfully start a new game! Game ID: {0}. # of tries remaining: {1}. Word: {2}.".format(game_id, tries_remains, word))
            while tries_remains>0:
                # get guessed letter from user code
                guess_letter = self.guess(word)
                    
                # append guessed letter to guessed letters field in hangman object
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print("Guessing letter: {0}".format(guess_letter))
                    
                try:    
                    res = self.request("/guess_letter", {"request":"guess_letter", "game_id":game_id, "letter":guess_letter})
                except HangmanAPIError:
                    print('HangmanAPIError exception caught on request.')
                    continue
                except Exception as e:
                    print('Other exception caught on request.')
                    raise e
               
                if verbose:
                    print("Sever response: {0}".format(res))
                status = res.get('status')
                tries_remains = res.get('tries_remains')
                if status=="success":
                    if verbose:
                        print("Successfully finished game: {0}".format(game_id))
                    return True
                elif status=="failed":
                    reason = res.get('reason', '# of tries exceeded!')
                    if verbose:
                        print("Failed game: {0}. Because of: {1}".format(game_id, reason))
                    return False
                elif status=="ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return status=="success"
        
    def my_status(self):
        return self.request("/my_status", {})
    
    def request(
            self, path, args=None, post_args=None, method=None):
        if args is None:
            args = dict()
        if post_args is not None:
            method = "POST"

        # Add `access_token` to post_args or args if it has not already been
        # included.
        if self.access_token:
            # If post_args exists, we assume that args either does not exists
            # or it does not need `access_token`.
            if post_args and "access_token" not in post_args:
                post_args["access_token"] = self.access_token
            elif "access_token" not in args:
                args["access_token"] = self.access_token

        time.sleep(0.2)

        num_retry, time_sleep = 50, 2
        for it in range(num_retry):
            try:
                response = self.session.request(
                    method or "GET",
                    self.hangman_url + path,
                    timeout=self.timeout,
                    params=args,
                    data=post_args,
                    verify=False
                )
                break
            except requests.HTTPError as e:
                response = json.loads(e.read())
                raise HangmanAPIError(response)
            except requests.exceptions.SSLError as e:
                if it + 1 == num_retry:
                    raise
                time.sleep(time_sleep)

        headers = response.headers
        if 'json' in headers['content-type']:
            result = response.json()
        elif "access_token" in parse_qs(response.text):
            query_str = parse_qs(response.text)
            if "access_token" in query_str:
                result = {"access_token": query_str["access_token"][0]}
                if "expires" in query_str:
                    result["expires"] = query_str["expires"][0]
            else:
                raise HangmanAPIError(response.json())
        else:
            raise HangmanAPIError('Maintype was not text, or querystring')

        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.code = None
        try:
            self.type = result["error_code"]
        except (KeyError, TypeError):
            self.type = ""

        try:
            self.message = result["erzror_description"]
        except (KeyError, TypeError):
            try:
                self.message = result["error"]["message"]
                self.code = result["error"].get("code")
                if not self.type:
                    self.type = result["error"].get("type", "")
            except (KeyError, TypeError):
                try:
                    self.message = result["error_msg"]
                except (KeyError, TypeError):
                    self.message = result

        Exception.__init__(self, self.message)

# API Usage Examples

## To start a new game:
1. Make sure you have implemented your own "guess" method.
2. Use the access_token that we sent you to create your HangmanAPI object. 
3. Start a game by calling "start_game" method.
4. If you wish to test your function without being recorded, set "practice" parameter to 1.
5. Note: You have a rate limit of 20 new games per minute. DO NOT start more than 20 new games within one minute.

In [153]:
api = HangmanAPI(access_token="ec0f6f19b61d33fe37d8343edbca3e", timeout=2000)

## Playing practice games:
You can use the command below to play up to 100,000 practice games.

In [157]:
for i in range(570):
    api.start_game(practice=1,verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))

Successfully start a new game! Game ID: 9def1bdf63ef. # of tries remaining: 6. Word: _ _ _ _ _ _ _ _ _ _ .
Guessing letter: e
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e _ _ e _ _ _ '}
Guessing letter: r
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ e r r e _ _ _ '}
Guessing letter: n
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ n _ e r r e _ _ n '}
Guessing letter: i
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 6, 'word': 'i n _ e r r e i _ n '}
Guessing letter: t
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 6, 'word': 'i n t e r r e i _ n '}
Guessing letter: o
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ongoing', 'tries_remains': 5, 'word': 'i n t e r r e i _ n '}
Guessing letter: a
Sever response: {'game_id': '9def1bdf63ef', 'status': 'ong

## Playing recorded games:
Please finalize your code prior to running the cell below. Once this code executes once successfully your submission will be finalized. Our system will not allow you to rerun any additional games.

Please note that it is expected that after you successfully run this block of code that subsequent runs will result in the error message "Your account has been deactivated".

Once you've run this section of the code your submission is complete. Please send us your source code via email.

In [159]:
for i in range(1000):
    print('Playing ', i, ' th game')
    # Uncomment the following line to execute your final runs. Do not do this until you are satisfied with your submission
    api.start_game(practice=0,verbose=False)
    
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)

Playing  0  th game
Playing  1  th game
Playing  2  th game
Playing  3  th game
Playing  4  th game
Playing  5  th game
Playing  6  th game
Playing  7  th game
Playing  8  th game
Playing  9  th game
Playing  10  th game
Playing  11  th game
Playing  12  th game
Playing  13  th game
Playing  14  th game
Playing  15  th game
Playing  16  th game
Playing  17  th game
Playing  18  th game
Playing  19  th game
Playing  20  th game
Playing  21  th game
Playing  22  th game
Playing  23  th game
Playing  24  th game
Playing  25  th game
Playing  26  th game
Playing  27  th game
Playing  28  th game
Playing  29  th game
Playing  30  th game
Playing  31  th game
Playing  32  th game
Playing  33  th game
Playing  34  th game
Playing  35  th game
Playing  36  th game
Playing  37  th game
Playing  38  th game
Playing  39  th game
Playing  40  th game
Playing  41  th game
Playing  42  th game
Playing  43  th game
Playing  44  th game
Playing  45  th game
Playing  46  th game
Playing  47  th game
Pl

HangmanAPIError: {'error': 'You have reached 1000 of games', 'status': 'denied'}

## To check your game statistics
1. Simply use "my_status" method.
2. Returns your total number of games, and number of wins.

In [161]:
[total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
success_rate = total_recorded_successes/total_recorded_runs
print('overall success rate = %.3f' % success_rate)

overall success rate = 0.544
